# Material de apoio

1. An Introduction to Empirical Legal Research. Epstein & Martin. Parte IV (introdução) Capítulo 9, item 9.3 - Logistic Regression.
2. Blog post [A Simple Interpretation of Logistic Regression Coefficients](https://towardsdatascience.com/a-simple-interpretation-of-logistic-regression-coefficients-e3a40a62e8cf)
3. Interpretable Machine Learning. MOLNAR, Christoph. [Logistic Regression: https://christophm.github.io/interpretable-ml-book/logistic.html](https://christophm.github.io/interpretable-ml-book/logistic.html)

As 2 últimas referências contém um pouco de notação matemática e equações que representam o modelo logístico. Contudo, a ideia principal do texto pode ser compreendida mesmo sem se preocupar em entender essas fórmulas e abstraindo seu significado.

O principal conteúdo dessas referências é como podemos expressar e interpretar alguns elementos da regressão logística.

# Regressões logísticas

Nas últimas aulas aprendemos como averiguar e testar a existência de relações lineares entre variáveis. Quando começamos, nossa principal preocupação era de entender a relação entre 2 variáveis de um mesmo conjunto de observações. Utilizando o **teste de Pearson** podemos verificar *se há uma relação linear entre as 2 variáveis no nosso conjunto de dados*. Além disso, utilizando **regressões** podemos não apenas checar se há correlação entre as variáveis, mas também *estimar o quanto as variáveis variam conjuntamento*, isto é, quanto Y tende a variar com uma variação de uma unidade em X. Utilizando essa ferramenta somos capazes de quantificar na unidade medida a relação entre as variáveis, ou mesmo prever o valor da variável dependente a partir de outras variáveis independentes, que acreditamos estar correlacionadas.

Uma característica importante das regressões lineares é que elas são usadas para lidar com variáveis dependentes com valores **contínuos**, por exemplo, valor de uma condenação ou tempo decorrido até uma decisão. No exemplo das aulas anteriores, estávamos trabalhando com a média dos julgamentos de diversas pessoas sobre violação a regras em uma escala de 7 pontos, uma variável que podia assumir qualquer valor entre 1 e 7 para cada um dos casos.

Contudo, muitas vezes estamos interessados em variáveis com características diferentes. Inclusive no direito. Podemos desejar verificar se um caso será concedido ou não, se um juiz de colegiado irá votar a favor ou contra nosso pedido, se haverá um pedido de vista no caso, classificar o tema de um documento.

Nesses casos, ao invés de tratarmos com variáveis contínuas, estamos trabalhando com **variáveis discretas** ou **categóricas**. Queremos saber, por exemplo, se um determinado processo vai ser julgado como procedente ou improcedente, ou qual vai ser o resultado exato dentre um conjunto finito de categorias. Esse tipo diferente de dado exige um tipo diferente de modelo estatística. Em especial, quando tratamos de variáveis dependentes **binárias**, podemos categorizar a variável que estamos interessado como 1 ou 0, podemos usar regressões logísticas.

Neste notebook, vamos usar regressões logísticas para reproduzir algumas análises do artigo de [Posner e De Figueiredo que avalia a existência de vieses na Corte Internacional de Justiça.](https://chicagounbound.uchicago.edu/law_and_economics/290/). Para isso, vamos tentar entender se alguma das variáveis estão associadas a maior chance de cada Juiz votar a favor do país que fez a petição na Corte.

O [dicionário de dados com a explicação do significado de cada variável dos dados está disponível como suplemento ao livro An Introduction to Empirical Legal Research](http://empiricallegalresearch.org/assets/Codebook_icj.pdf), que indicamos como material ao longo do curso. Foram feitas algumas adaptações no dataset para manter o significado de algumas colunas mais próximo de como estamos acostumados a lidar quando pensamos sobre o problema, do que a forma representada no dataset original que usa númeração para todas as categorias.

No fim do material e na próxima aula, veremos como esse mesmo problema pode ser generalizado como uma tarefa de classificação ao qual podemos aplicar diversos métodos de aprendizado de máquina.

## Carregando os dados

Para falar sobre o padrão de voto dos juízes da ICJ vamos carregar os dados que estão disponibilizados em um link específico

In [1]:
import pandas as pd

In [2]:
icj = pd.read_csv("https://bit.ly/dataset-ICJ")

icj.head()

,caseNumber,caseName,respEU,respGDP,respLegalTrad,decision1,judgeEU,judgeGDP,judgeLegalTrad,appEU,appGDP,appLegalTrad,jurisBasis
0,C0.5,Corfu Channel (Preliminary Objection),0.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,NaN,1.0,3.0
1,C1,Corfu Channel,0.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,NaN,1.0,3.0
2,C1.5,Corfu Channel (Assessment of Compensation),0.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,NaN,1.0,3.0
3,C56,Certain Phosphate Lands in Nauru,0.0,19573.818359,1.0,1.0,0.0,4209.721191,NaN,0.0,NaN,NaN,3.0
4,C59,East Timor,0.0,21771.312500,1.0,0.0,0.0,4245.033691,NaN,1.0,12906.996094,2.0,3.0


Algumas informações básicas sobre as variáveis e o que desejamos fazer:

1. **caseNumber** identifica as decisões *individualmente*. Perceba pelas primeiras informações de caseName que nossa unidade é voto em decisão, podendo um mesmo processo conter múltiplas observações de decisão, e a mesma decisão conter múltiplos votos de juízes diferentes.
2. As medidas de PIB são em dólares americanos per capita.

Nossa tarefa: Estamos interessados em analisar *se os juízes com certas características tendem a votar mais em favor do aplicante (país que apresentou a petição) nos casos levados a corte*, essa informação está representada na coluna `decision1`.

Como discutimos, regressões logísticas têm uma variável binária como variável dependente. Qual é a variável dependente da análise proposta? Ela é binária?

Lembrando, para checar podemos usar o método `value_counts()`

In [3]:
icj["decision1"].value_counts()

decision1
1.0    754
0.0    748
Name: count, dtype: int64

Vamos substituir alguns dos valores de representação numérica para dar nome as categorias expressadas.

In [4]:
icj.replace(
    to_replace={
        "appLegalTrad": {
            1: "english",
            2: "french",
            3: "german",
            4: "scandinavian",
            5: "socialist",
        },
        "respLegalTrad": {
            1: "english",
            2: "french",
            3: "german",
            4: "scandinavian",
            5: "socialist",
        },
        "judgeLegalTrad": {
            1: "english",
            2: "french",
            3: "german",
            4: "scandinavian",
            5: "socialist",
        },
        "jurisBasis": {
            1: "treaty",
            2: "special agreement",
            3: "special clause",
        },
    },
    inplace=True,
)

icj.head(3)

,caseNumber,caseName,respEU,respGDP,respLegalTrad,decision1,judgeEU,judgeGDP,judgeLegalTrad,appEU,appGDP,appLegalTrad,jurisBasis
0,C0.5,Corfu Channel (Preliminary Objection),0.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,NaN,english,special clause
1,C1,Corfu Channel,0.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,NaN,english,special clause
2,C1.5,Corfu Channel (Assessment of Compensation),0.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,NaN,english,special clause


A sintaxe das regressões logísticas é a mesma com a sintaxe que usamos para expressas as regressões lineares. Vamos escrever uma `string` começando com o nome de nossa variável dependente (Y), aproximada (`"~"`) pela combinação das variáveis independentes (X). Devemos, entretanto, mudar a função usada de `ols` para `logit`.

Vamos importar a função abaixo e criar um primeiro modelo que prevê a decisão do juiz apenas do PIB do seu país de origem:

In [5]:
from statsmodels.formula.api import logit
model = logit("decision1 ~ judgeGDP", data=icj).fit()

model.summary()

Optimization terminated successfully.
         Current function value: 0.687987
         Iterations 4


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              decision1   No. Observations:                 1146
Model:                          Logit   Df Residuals:                     1144
Method:                           MLE   Df Model:                            1
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                0.007181
Time:                        17:08:18   Log-Likelihood:                -788.43
converged:                       True   LL-Null:                       -794.14
Covariance Type:            nonrobust   LLR p-value:                 0.0007322
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.1330      0.078      1.705      0.088      -0.020       0.286
judgeGDP   -2.292e-05   6.85e-06     -3.346      0.001   -3.63e-05   -9.49e-06
==============================================================================
"""

A apresentação do resultado é um pouco diferente ao que vimos com as regressões lineares. Mas temos os principais itens que interpretamos repetidos, mas com ligeiramente diferentes. O valor-p é ainda um indicador de significância estatística, e tem a mesma interpretação que já estabelecemos desde a introdução de testes estatísticos. Ja o coeficiente indica a direção e intensidade da relação entre as variáveis, e o Pseudo-R mede quão bem o modelo se ajusta aos dados, mas a interpretação do valor observado é diferente. Vamos tratar disso mais adiante.

Quanto ao resultado material do modelo criado, como esperado, ele não é muito útil para descrever o comportamento dos juízes. Apesar de haver uma relação significativa entre o PIB do país de origem dos juízes e a decisão deles, o Pseudo-R² é extremamente baixo, indicando que o modelo não se ajusta bem aos dados.

Também podemos gerar o output como texto, e printá-lo para leitura.

In [6]:
print(model.summary().as_text())

                           Logit Regression Results                           
Dep. Variable:              decision1   No. Observations:                 1146
Model:                          Logit   Df Residuals:                     1144
Method:                           MLE   Df Model:                            1
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                0.007181
Time:                        17:08:18   Log-Likelihood:                -788.43
converged:                       True   LL-Null:                       -794.14
Covariance Type:            nonrobust   LLR p-value:                 0.0007322
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.1330      0.078      1.705      0.088      -0.020       0.286
judgeGDP   -2.292e-05   6.85e-06     -3.346      0.001   -3.63e-05   -9.49e-06


Antes de adentrar na interpretação do modelo, vamos primeiro criar um modelo melhor usando mais das variáveis que temos disponíveis.

## Regressões logísticas múltiplas

Assim como no caso das regressões lineares, também podemos usar várias variáveis independentes para investigar nossa variável dependente.

Vamos olhar para alguns candidatos potenciais:

A tradição jurídica do país de origem do juiz.

In [7]:
icj.judgeLegalTrad.value_counts()

judgeLegalTrad
french          523
english         349
socialist       267
german          114
scandinavian     45
Name: count, dtype: int64

A tradição jurídica do país de origem do respondente.

In [8]:
icj.respLegalTrad.value_counts()

respLegalTrad
english         694
french          427
scandinavian    163
socialist        16
german           15
Name: count, dtype: int64

Se o país respondente é membro da União Europeia.

In [9]:
icj.respEU.value_counts()

respEU
0.0    1245
1.0     310
Name: count, dtype: int64

Vamos especificar um modelo que considere não apenas a renda do país de origem do Juiz, mas também o tipo de norma que fornece competência de jurisdição para a Corte, assim como se o respondente é membro da União Européia.

In [10]:
model = logit("decision1 ~ judgeGDP + respEU + jurisBasis", data=icj).fit()

print(model.summary().as_text())

Optimization terminated successfully.
         Current function value: 0.675323
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:              decision1   No. Observations:                 1146
Model:                          Logit   Df Residuals:                     1141
Method:                           MLE   Df Model:                            4
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                 0.02546
Time:                        17:08:21   Log-Likelihood:                -773.92
converged:                       True   LL-Null:                       -794.14
Covariance Type:            nonrobust   LLR p-value:                 3.527e-08
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                       -0.5894      0.213     -2.761      0.006

## Pausa para reflexão: o que queremos medir/analisar?

Poderíamos desenvolver ainda mais esse modelo, adicionando mais variáveis. Mas antes, vamos pausar e pensar sobre o que queremos avaliar com nossa regressão, como podemos interpretá-la e relacionar com o problema concreto.

Em primeiro lugar, a hipótese de Posner e De Figueiredo sobre o comportamento dos juízes avaliada nos artigos originais diz a respeito da **diferença** entre a renda do país de origem do peticionante e do juíz. Nós temos essas informações no momento?

Mas com as informações a nossa disposição, poderíamos gerá-la? Como?

Por outro lado, queremos avaliar, por exemplo, se juízes julgam de forma diferente por serem da União Européia, ou estamos mais preocupados se juízes tendem a julgar a favor de países semelhantes ao de sua origem? Uma forma que podemos pensar sobre o problema é: se tanto o juiz quanto uma das partes são da União Europeia, isso pode afetar seu comportamento?

Ambas as perguntas são válidas e distintas, mas devemos usar variáveis diferentes para tentar capturar cada uma delas.

Podemos fazer o mesmo questionamento sobre a tradição jurídica das partes e dos juizes.

A título de exemplo, vamos criar uma nova coluna que indica se a tradição jurídica do país que levou o caso a corte e do juiz são iguais. Isso pode ser feito de inúmeras maneiras.


In [11]:
# opção 1: nativo do pandas
# A linha abaixo retornau ma coluna informando se as duas variáveis são iguais
# Teste executá-la em outra célula
(icj["appLegalTrad"] == icj["judgeLegalTrad"]).astype(int)
# lembrando: as operações por padrão são feitas linha a linha no pandas. Após, podemos interpretar o booleano como int.
icj['trad_equals_pet'] = (icj["appLegalTrad"] == icj["judgeLegalTrad"]).astype(int)

icj['trad_equals_pet'].value_counts()


trad_equals_pet
0    1308
1     252
Name: count, dtype: int64

Poderíamos obter o mesmo resultado usando uma iteração sobre o DataFrame:

In [12]:
# opção 2: iteração
pet_trad_equals_judge = list()

for _indice, row in icj.iterrows():
    if row["appLegalTrad"] == row["judgeLegalTrad"]:
        pet_trad_equals_judge.append(1)
    else:
        pet_trad_equals_judge.append(0)

print(pet_trad_equals_judge[:5])

icj['trad_equals_pet'] = pet_trad_equals_judge

icj['trad_equals_pet'].value_counts()

[0, 0, 0, 0, 0]


trad_equals_pet
0    1308
1     252
Name: count, dtype: int64

Vamos calcular também a diferença entre o PIB dos países partes.

In [13]:
icj["parties_GDP_dif"] = (icj["respGDP"] - icj["appGDP"])

icj["parties_GDP_dif"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 1560 entries, 0 to 1559
Series name: parties_GDP_dif
Non-Null Count  Dtype  
--------------  -----  
854 non-null    float64
dtypes: float64(1)
memory usage: 12.3 KB


Vamos criar um novo modelo usando essas informações

In [14]:
model = logit("decision1 ~ trad_equals_pet + parties_GDP_dif + judgeGDP + jurisBasis", data=icj).fit()

print(model.summary().as_text())

Optimization terminated successfully.
         Current function value: 0.669438
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:              decision1   No. Observations:                  734
Model:                          Logit   Df Residuals:                      728
Method:                           MLE   Df Model:                            5
Date:                Mon, 06 Jan 2025   Pseudo R-squ.:                 0.03408
Time:                        17:08:29   Log-Likelihood:                -491.37
converged:                       True   LL-Null:                       -508.70
Covariance Type:            nonrobust   LLR p-value:                 1.752e-06
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                       -0.9552      0.287     -3.323      0.001

Usando o Pseudo-R², podemos ver que esse modelo é ligeiramente melhor ao que fizemos antes de criar nossas novas variáveis. Por outro lado, essas novas variáveis fornecem informações interessantes sobre fatores do caso que podem influenciar o comportamento dos juízes.

Verifique quais das variáveis usadas são estatisticamente significativas.

Sob o ponto de vista que avaliaríamos uma regressão linear, esse modelo poderia ainda ser considerado ruim. É importante ressaltar que a interpretação não deve ser a mesma! Vamos ver isso em seguida.


# Avaliando regressões logísticas

Como mencionamos, podemos usar a mesma lógica que usamos para interpretar os resultados de uma regressão linear para interpretar os resultados de uma regressão logística, mas nenhum dos parâmetros funciona exatamente da mesma forma. Por exemplo, o resultado não possui um *R²*, mas sim *um Pseudo-R²*, que tenta aproximar a mesma lógica que vimos no caso da regressão linear simples.

É importante considerarmos também o valor de P de cada uma das variáveis, para saber se o efeito delas é significativo.

Essa é uma forma de pensarmos sobre os resultados da regressão logística, alinhada com o que vimos até aqui. Podemos chamar essa visão de *"estatística"*. Ela é útil quando queremos olhar para cada uma das variáveis e sua relação com o fenônomeno sob análise.

Mas e se quisermos avaliar o resultado do nosso modelo como um todo? Ou se queremos usar nosso modelo para prever casos futuros, como podemos avaliar quão bem ele é capaz de prever casos?

Uma maneira mais intuitiva de avaliar o quão boa ou ruim é uma regressão logística é saber *quantas vezes ela acerta a previsão a respeito do resultado*: assim como podemos usar uma regressão linear para saber qual é o valor esperado na variável dependente dado certos valores nas variáveis independentes, podemos fazer o mesmo no caso da regressão logística.  A diferença é que agora a regressão logística irá retornar a *probabilidade esperada de que o valor seja 1*.

Essa forma de pensar está normalmente associada a prática de aprendizado de máquinas. 

Vamos usar o mesmo método `predict` que vimos em regressão linear com as colunas relevantes.

In [15]:
icj[["trad_equals_pet", "parties_GDP_dif", "judgeGDP", "jurisBasis"]].head()

,trad_equals_pet,parties_GDP_dif,judgeGDP,jurisBasis
0,0,NaN,NaN,special clause
1,0,NaN,NaN,special clause
2,0,NaN,NaN,special clause
3,0,NaN,4209.721191,special clause
4,0,8864.316406,4245.033691,special clause


Um problema inicial é que temos uma alta quantidade de valores não preenchidos `NaN`. Para lidar com isso vamos separar apenas as linhas em que essa informação está preenchida.

In [29]:
df_previsao = icj.dropna(
    subset=["trad_equals_pet", "parties_GDP_dif", "judgeGDP", "jurisBasis", "decision1"]
).copy()


In [30]:
probabilidade_de_sucesso = model.predict(df_previsao[["trad_equals_pet", "parties_GDP_dif", "judgeGDP", "jurisBasis"]])

probabilidade_de_sucesso.head(2)

4     0.487670
11    0.474677
dtype: float64

Vamos agora adicionar o valor previsto ao dataframe

In [31]:
df_previsao["probabilidade_de_sucesso"] = probabilidade_de_sucesso

df_previsao.head(2)

,caseNumber,caseName,respEU,respGDP,respLegalTrad,decision1,judgeEU,judgeGDP,judgeLegalTrad,appEU,appGDP,appLegalTrad,jurisBasis,trad_equals_pet,parties_GDP_dif,probabilidade_de_sucesso
4,C59,East Timor,0.0,21771.312500,english,0.0,0.0,4245.033691,NaN,1.0,12906.996094,french,special clause,0,8864.316406,0.487670
11,C69,Fisheries Jurisdiction,0.0,24944.189453,english,1.0,0.0,4540.739258,NaN,1.0,17329.433594,french,special clause,0,7614.755859,0.474677


Alternativamente, podemos fazer com que a nossa regressão logística *classifique* cada uma das observações como um 1 ou como um 0. Para isso, podemos implementar a seguinte heurística: se a probabilidade prevista de que a ação seja julgada procedente for menor do que 50%, vamos tratar a previsão como se fosse de derrota (0); por outro lado, se a probabilidade de que a ação seja julgada procedente for maior do que 50%, vamos tratar a previsão como sendo de vitória (1).

Vamos usar a função `eval` para criar uma nova coluna usando essa heurística:

In [32]:
df_previsao["prev"] = df_previsao.eval("probabilidade_de_sucesso > 0.5")

Com as métricas que calculamos, podemos saber qual é o percentual de acerto do nosso modelo ao o voto dos juízes.

Chamamos essa medida de *acurácia* do modelo. Ela é a forma mais intuitiva de avaliar o quão bom é um modelo de classificação. Falaremos de outras adiante.

In [35]:
df_previsao["acerto_bin"] = df_previsao["decision1"] == df_previsao["prev"]
df_previsao["acerto_bin"] = df_previsao["acerto_bin"].astype("int")
df_previsao["acerto_bin"].mean()

0.5980926430517711

Isso significa que estamos acertando cerca de 56% das previsões sobre voto dos juízes. Isso é bastante impressionante, afinal estamos prevendo os votos de casos altamente complexos. Não?

Por outro lado, a métrica de percentual de acerto possui algumas limitações.

Vocês conseguem pensar em algumas delas?

### Desafio

Por que usamos o valor 0.5 em nossa heurística? Poderíamos alterar esse valor? Qual seria a consequência de alterá-lo?

Veja o valor médio da coluna com as decisões. Como devemos interpretar o percentual de acerto a partir disso?

In [40]:
df_previsao["decision1"].mean()

0.49318801089918257

In [37]:
df_previsao["acerto_bin"].mean()

0.5980926430517711

## Limitações do percentual de sucesso

Uma primeira limitação que precisamos enfrentar é a seguinte: um modelo não é muito bom se tudo o que ele consegue fazer é aproximar algum chute tosco.

Se simplesmente jogassemos uma moeda para cima quantos casos esperaríamos acertar? Essa pergunta é um pouco mais complexa do que parece de início!

Podemos escolher chutar que todos os juízes semrpe vão tomar um mesmo tipo de decisão, por exemplo a favor do réu. Nesse caso, conseguiríamos acertar quase 50% dos casos: 49,3% para ser preciso, conforme a média na célula acima.

Mas não é necessário saber muito sobre ciência de dados para desenvolver essa heurística.

Vamos ver quantas vezes prevemos cada tipo de voto:

In [50]:
df_previsao["prev"].value_counts()

prev
True     373
False    361
Name: count, dtype: int64

Podemos ver que não se assemalha a uma heurística tão tosca quanto essa, afinal ele apresenta um número balanceado de predição das duas classes.

Por outro lado, isso parece indicar que provavelmente ele erra uma quantidade razoável de vezes nos 2 casos. Vamos verificar:

In [41]:
df_previsao.query("decision1 == 1")["acerto_bin"].value_counts()

acerto_bin
1    220
0    142
Name: count, dtype: int64

In [42]:
df_previsao.query("decision1 == 0")["acerto_bin"].value_counts()

acerto_bin
1    219
0    153
Name: count, dtype: int64

Uma forma de visualizar não só o % de acerto, mas também qual foi o % de acerto em cada uma das categorias e, portanto, conhecer os % de **falsos positivos**, **verdadeiros positivos**, **falsos negativos** e **verdadeiros negativos** de um modelo preditivo é usar uma matriz de confusão:

In [44]:
from sklearn.metrics import confusion_matrix

print(confusion_matrix(df_previsao["decision1"], df_previsao["prev"]))

[[219 153]
 [142 220]]


Essa matriz não fica muito fácil de ler. O código abaixo deixa as coisas um pouco mais claras:

In [45]:
pd.DataFrame(columns = [0, 1], index = [0,1], data=confusion_matrix(df_previsao["decision1"], df_previsao["prev"]))

,0,1
0,219,153
1,142,220


O modelo parece performar de forma semelhante tanto quando os juízes votam a favor do aplicante quanto quando votam contra. Isso é um bom sinal, pois indica que o modelo não está simplesmente chutando um resultado. Contudo, isso nem sempre é o caso.

Imagine que se chutarmos para todos os casos de um tribunal que ele não será julgado pelo STF, acertaríamos a imensa maioria dos casos. Mas também erraríamos 100% dos casos que seriam levados até a corte.

A partir dos números obtidos, podemos calcular estatísticas importantes. Por exemplo, podemos saber qual é a proporção de verdadeiros positivos e de verdadeiros negativos:

In [48]:
verdadeiro_positivo = 220 / (220 + 142) # probabilidade de ser positivo dado que a previsão foi positiva (classificada como 1)
verdadeiro_negativo = 219 / (219 + 153) # probabilidade de ser negativo dado que a previsão foi negativa

print(f"A taxa de verdadeiro positivo é {round(verdadeiro_positivo, 4) * 100}%")
print(f"A taxa de verdadeiro negativo é {round(verdadeiro_negativo, 4) * 100}%")

A taxa de verdadeiro positivo é 60.77%
A taxa de verdadeiro negativo é 58.87%


Várias outras métricas podem ser calculadas usando o método `classification_report`:

In [49]:
from sklearn.metrics import classification_report

print(classification_report(df_previsao["decision1"], df_previsao["prev"]))

              precision    recall  f1-score   support

         0.0       0.61      0.59      0.60       372
         1.0       0.59      0.61      0.60       362

    accuracy                           0.60       734
   macro avg       0.60      0.60      0.60       734
weighted avg       0.60      0.60      0.60       734



Vamos explicá-las abaixo:

A **precisão** representa o percentual de verdadeiros positivos entre todas as observações consideradas como positivas pelo modelo para aquela classe. Ou seja, considera os casos de certa classe que o modelo acertou em relação a todos os casos que ele previu serem daquela classe.

$$ Precisão = \frac {Verdadeiros~Positivos}{Verdadeiros~Positivos + Falsos~Positivos} $$

Assim, dizer que a precisão do modelo com relação ao voto 0 foi de 61% significa que 61% das vezes em que o modelo previu que o voto do juiz seria contra o peticionante, o voto de fato foi contra. A mesma interpretação pode ser dada com relação ao resultado 1. Nesse caso, a precisão significa a mesma coisa do que a taxa de verdadeiros positivos.

O **recall** indica o percentual de membros existentes daquela classe que foram corretamente identificados como membros da classe:

$$ Recall = \frac {Verdadeiros~Positivos}{Verdadeiros~Positivos + Falsos~Negativos} $$

Assim, dizer que o modelo teve recall de 59% com relação ao resultado 0 significa que identificamos 59% dos votos contra o peticionante como sendo de fato contrário a seu interesse. Comparar recall e precisão nos diz coisas importantes sobre o modelo.

Finalmente, o **f1 score** é uma métrica que leva em conta tanto o recall quanto a precisão.

$$ F1 = 2 * \frac{Precision * Recall}{Precision + Recall} $$

Valores mais altos são melhores, mas é difícil interpretar os resultados de maneira absoluta. Para comparação, é interessante ver a média ponderada ("weighted") entre os f1 scores das duas classes, no caso do nosso modelo, 0.60.

Podemos usar essas métricas agora para comparar diferentes modelos e para saber se o nosso modelo é bom ou ruim de uma maneira geral.

O quão bom seria esse modelo para prever o voto dos juízes? 


## Interpretando regressão logística como modelo estatístico.

Mas e se estivermos interessados em saber informações sobre uma das variáveis que incluímos em nossa logit? Poderíamos fazer isso com nossa "mentalidade" de predição que usamos acima. Poderíamos comparar o modelo com e sem um parâmetro e ver o quão melhor ele fica, e refletir sobre qual seria sua relação com o resultado sendo previsto.

Outra opção é voltarmos à perspectiva estatística que usamos na regressão linear. Vamos olhar o sumário da regressão novamente.

In [ ]:
model = logit("decision1 ~ trad_equals_pet + parties_GDP_dif + judgeGDP + jurisBasis", data=icj).fit()

print(model.summary().as_text())

Optimization terminated successfully.
         Current function value: 0.669438
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:              decision1   No. Observations:                  734
Model:                          Logit   Df Residuals:                      728
Method:                           MLE   Df Model:                            5
Date:                Wed, 10 Jul 2024   Pseudo R-squ.:                 0.03408
Time:                        15:05:47   Log-Likelihood:                -491.37
converged:                       True   LL-Null:                       -508.70
Covariance Type:            nonrobust   LLR p-value:                 1.752e-06
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                       -0.9552      0.287     -3.323      0.001

Nas aulas anteriores falamos de algumas estatísticas pelas quais podemos iniciar nossa interpretação do modelo. Isso também vale aqui, mas a forma como vamos conectá-las com o resultado é diferente.

No material de apoio há algumas referências destacadas de como procedor com modelos logísticos.

Quanto ao modelo como um todo, nossa opção além do que fizemos de comparar a previsão é analisar o Pseudo-R², lembrando que essa estatística possui limitações em especial se queremos compararmos performances de modelos treinados em conjuntos de dados diferentes.

E quanto a cada parâmetro individualmente? Digamos que queremos analisar o impacto do PIB per capita do país de origem dos juízes no modelo reportado acima. Como devemos fazê-lo?

Assim como nos demais casos, o local mais comum de se iniciar é pelo valor P. Ele tem a mesma interpretação que nos casos anteriores, inclusive as limitações.

Agora, queremos comparar o impacto do parâmetro individual no resultado. Na regressão linear costumamos interpretar a alteração de uma unidade da variável independente como a variação esperada a ser observada na variável dependente, mantendo-se todo  o mais constante. Contudo, isso não faz sentido no caso da regressão logística! Por quê?

Vamos lembrar do resultado de quando usamos o método `predict` com nosso modelo:

In [29]:
probabilidade_de_sucesso.iloc[:5]

4     0.487670
11    0.474677
14    0.424945
17    0.397522
18    0.402168
dtype: float64

Como o nome da variável indica, geralmente interpretamos o resultado de uma regressão logística como a probabilidade da variável independente ocorrer naquele caso (codificada como 1).

Assim, se voltarmos a tabela e olharmos o coeficiente do que concedeu jurisdição para a corte, $jurisBasis[T.treaty]$ tem um coeficiente de 1.27 e seu valor P indica uma relação significativa, mas aumentar a probabilidade nesse montante indicaria mais de 100% de chance!

O coeficiente logística não tem essa interpretação direta. Vamos ver a equação que representa esse tipo de modelo:

$$p(x)=\frac{1}{1+e^{-(\beta_0+\beta_1 x+ ... + \beta_n x_n)}}$$

Essa equação é bem menos familiar e mais confusa que a da regressão linear. O intercepto da regressão logística é o $\beta_0$ que está em um expoente no denominador da fração, e os coeficientes sao $\beta_1$ e demais.

O importanto é que podemos converter esse coeficiente para algo próximo de como pensamos em probabilidade: em inglês essa interpretação é denominada de odd ratio, algo semelhante a razão de chances ou razão de probabilidade. Esse tipo de resultado é muito usado para expressar apostas e jogos de azar. 

Assim como na regressão linear costumamos formular nossa explicação no aumento de uma unidade da variável.

Para converter o valor para odd ratio precisamos fazer uma operação matemática com o coeficiente e comparar a probabilidade para diferentes valores da variável independente de interesse. Vamos usar jurisBasis como exemplo.

Lembrando: como jurisBasis é uma variável categórica, ela é calculada comparando a outro valor da mesma variável. No caso, como o valor que não aparece na tabela é 'special agreement' esse é o valor de comparação. Vamos comparar com quando a jurisdição vem de um tratado.

In [30]:
icj.jurisBasis.unique()

array(['special clause', 'special agreement', 'treaty', nan], dtype=object)

Podemos calcular a fração entre a probabilidade de quando a base para jurisdição é tratado e cláusula especial. Algo como:


$$Odds Ratio = \frac{P(jurisBasis=treaty)}{P(jurisBasis=special\ agreement)}$$

Para calcular a probabilidade precisáriamos resolver a equação estranha da regressão logística... Felizmente podemos usar `python` para nos auxiliar e calcular facilmente a Odd Ratio para todos os coeficientes:

Você não precisa se preocupar com o código a seguir, basta saber que ele está calculando a odd ratio para cada uma das variáveis independentes do modelo.

In [59]:
np.exp(model.params["parties_GDP_dif"]*10000)

1.5095505774384976

In [52]:
import numpy as np

# Vamos criar um dataframe com as informações
# primeiro, precisamos resolver o log natural dos coeficientes. Vamos usar o numpy
model_odds = pd.DataFrame(np.exp(model.params), columns= ['odd_ratio'])

# adicionando o valor p
model_odds['p-value']= model.pvalues

# adicionando intervalo de confiança
model_odds[['2.5%', '97.5%']] = np.exp(model.conf_int())

model_odds

,odd_ratio,p-value,2.5%,97.5%
Intercept,0.384720,0.000889,0.219023,0.675772
jurisBasis[T.special clause],1.732106,0.056278,0.985406,3.044625
jurisBasis[T.treaty],3.576664,0.000030,1.966623,6.504818
trad_equals_pet,1.529644,0.017602,1.076926,2.172677
parties_GDP_dif,1.000041,0.000340,1.000019,1.000064
judgeGDP,0.999998,0.850445,0.999977,1.000019


Agora podemos interpretar nossos coeficientes!

Nossa nova tabela diz que a razão de chance quando a Jurisdição é tratado é de 3.57. Isso significa que a probabilidade prevista para concessão do pedido nesses caso e 3.57 vezes maior que aquela atribuída a acordo especial, se mantermos todo o resto constante.

A título de exemplo, se a probabilidade para acordo fosse de 20%, a de tratado seria de 71.4%.

Já o aumento de 1 unidade na diferença do PIB per capita entre as partes leva a uma razão de 1.000041. Dada a escala de medição de PIB podemos formular isso em uma unidade de milha. Uma diferença positiva de US$1.000 no PIB per capita (1000 unidades) das partes (respondente subtraída do peticionante) é associada a probabilidade 1.042 vezes maior, e 1.5 vez para US$10.000.

**CUIDADO: a chance de probabilidade (odd ratio) não diz diretamente qual a probabilidade prevista, apenas o quão maior ela é em relação a um valor base.**